# AI Agents - Team 7
**Integrantes**
- Jerónimo Hoyos Botero (jhoyosbo@unal.edu.co)
- Felipe Garzon Melguizo (fgarzonm@unal.edu.co
- Bryan Steven Yama Cruz (byama@unal.edu.co)


In [ ]:
pip install chromadb sentence-transformers pandas


In [ ]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Step 1: Load your CSV file
csv_path = "movies.csv"  # Update if needed
df = pd.read_csv(csv_path)

# Check for "title" field
if "title" not in df.columns:
    raise ValueError("The CSV does not contain a 'title' column.")

/tmp/ipython-input-3947456998.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


## Exercise

- Now embed the 'overview' field, how much time took to do that?
- Try a couple of search examples and report.
- Try the very same movie title and compare these results with the ones of google (Universal sentence encoder)
- Compare the time of prediction of this approach and the one of google
  


# Solución ejercicio 3


## **Inciso (1)**
**Enunciado:** Now embed the 'overview' field, how much time took to do that?

**Resultado:** Tardó 1 minuto y 36 segundos

In [ ]:
# Step 2: Create embeddings using a SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2") # this is an embedder
texts = df["overview"].astype(str).tolist()
embeddings = model.encode(texts, show_progress_bar=True)


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Batches:   0%|          | 0/1421 [00:00<?, ?it/s]

In [ ]:
# Step 3: Set up Chroma DB and collection

from tqdm import tqdm

# Initialize Chroma client
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection("titles_collection")

# Batch insert (max 5000 per batch)
batch_size = 5000
for i in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[i:i+batch_size]
    batch_embeddings = embeddings[i:i+batch_size]
    batch_ids = [f"id_{j}" for j in range(i, i+len(batch_texts))]
    collection.add(
        documents=batch_texts,
        embeddings=batch_embeddings,
        ids=batch_ids
    )

print("All documents inserted in batches.")


100%|██████████| 10/10 [00:04<00:00,  2.45it/s]

All documents inserted in batches.


## **Inciso (2)**
**Enunciado:** Try a couple of search examples and repor

**Resultado:**

Usando la descripción de spiderman encontró con exito en el top 1 a que pelicula se referia.

In [ ]:
import time
results = collection.query(
    query_texts=["A teenager gets bitten by a spider and gets superpowers, but with great power comes great responsibility, and he still needs to learn that."],
    n_results=5,
    include=["documents", "distances"]  # Include relevance scores (lower is better)
)

print("Top Matches:")
for doc, score in zip(results["documents"][0], results["distances"][0]):
    probability = round(1 / (1 + score), 2)  # Convert distance to a pseudo-probability
    print(f"{doc}, confidence: {probability}")


Top Matches:
After being bitten by a genetically altered spider, nerdy high school student Peter Parker is endowed with amazing powers., confidence: 0.6
Teenagers from a small town and their high school science teacher join forces to battle a giant mutant spider, living in a cave nearby and getting hungry., confidence: 0.54
A teenager finds himself transported to an island where he must help protect a group of orphans with special powers from creatures intent on destroying them., confidence: 0.52
An 11-year-old boy becomes an unlikely superhero when he discovers that he has the ability to leave his body and fly through walls., confidence: 0.51
A boy is the only family member without superpowers in this Disney Film. The world depends on him saving his family from computerized brainwashers. Will he realize that it doesn't take superpowers to be a hero in time to help them defeat the villains?, confidence: 0.51


## **Inciso (3)**
**Enunciado:** Try the very same movie title and compare these results with the ones of google (Universal sentence encoder)

**Resultado:**

Usando la misma descripción de spiderman demostró el mismo top 1, pero en lo demás cambió.

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print("Done!")

Done!


In [ ]:
df = pd.read_csv("movies.csv", nrows=45000)

df['original_title'] = df['original_title'].str.lower()

df['overview'] = (
    df['overview']
    .fillna('')
    .astype(str)
    .str.lower()
)

/tmp/ipython-input-1420576903.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("movies.csv", nrows=45000)


In [ ]:

overviews = []

for index, row in df.iterrows():
    overviews.append(row.overview)


bank_vec = model(overviews)

mystr = 'A teenager gets bitten by a spider and gets superpowers, but with great power comes great responsibility, and he still needs to learn that.'
print(mystr)

query_vec = model([mystr])

correlation = np.inner(query_vec, bank_vec).flatten()

top_indices = np.argsort(correlation)[-5:][::-1]

print("\nTop five closest matches:")
for idx in top_indices:
    print(overviews[idx], correlation[idx])

top_matches = [(overviews[idx], correlation[idx]) for idx in top_indices]


A teenager gets bitten by a spider and gets superpowers, but with great power comes great responsibility, and he still needs to learn that.

Top five closest matches:
after being bitten by a genetically altered spider, nerdy high school student peter parker is endowed with amazing powers. 0.6842272
12-year-old pelle accidentally gets bitten by an ant and develops unimaginable superpowers. with help from his friend, comic book nerd wilhelm, pelle creates a secret identity as the superhero antboy. slowly things start to happen in the surburban community, and when a scary and crazy supervillain, flea, enters the scene, antboy must step up to the challenge.  the first danish superhero-movie. based on the books by kenneth bøgh andersen. 0.5692612
the seemingly invincible spider-man goes up against an all-new crop of villain – including the shape-shifting sandman. while spider-man’s superpowers are altered by an alien organism, his alter ego, peter parker, deals with nemesis eddie brock and 

## **Inciso (4)**
**Enunciado:** Compare the time of prediction of this approach and the one of google

**Resultado:**

Usando la vector database la predicción demor+o **4 segundos**

Mientras que usando la metodología del modelo de google demoró **10 segundos**

una diferencia de **6 segundos**